In [1]:
 pip install langchain_openai langchain_community langchain pymysql chromadb pyngrok flask -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
 import os
 os.environ["OPENAI_API_KEY"] = "sk-proj-J6yFWZCa46t4cURpxo6uT3BlbkFJ912Jhz3bwreeCyDQXEOi"

 db_user = "hackathon"
 db_password = "hackathon"
 db_host = "panosgio.org"
 db_name = "hackathon"
 from langchain_community.utilities.sql_database import SQLDatabase
 # db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
 db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
 #print(db.dialect)
 #print(db.get_usable_table_names())
 #print(db.table_info)


In [3]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
# question = 'Find how much revenue did we make from the top 10 customers and then sum the value of the 10 first only'
question = 'Can you list please the top six products?'
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": question})
# "what is price of `1968 Ford Mustang`"
print(query)

SELECT `Product Description`, `Revenue` 
FROM `LastWeekTopProducts` 
ORDER BY `Revenue` DESC 
LIMIT 6;


In [4]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

"[('Selmer Paris Reference 54 Alto', 409500), ('Keilwerth SX90R', 264000), ('Roland Fantom-8', 248500), ('Tama Starclassic Bubinga 5-Piece Shell Pack', 245000), ('Music Man John Petrucci Majesty', 240000), ('Nord Stage 3 Compact', 227500)]"

In [5]:
 from operator import itemgetter

 from langchain_core.output_parsers import StrOutputParser
 from langchain_core.prompts import PromptTemplate
 from langchain_core.runnables import RunnablePassthrough

 answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

 rephrase_answer = answer_prompt | llm | StrOutputParser()

 chain = (
     RunnablePassthrough.assign(query=generate_query).assign(
         result=itemgetter("query") | execute_query
     )
     | rephrase_answer
 )

 chain.invoke({"question": question})

'The top six products based on revenue are:\n1. Selmer Paris Reference 54 Alto - $409,500\n2. Keilwerth SX90R - $264,000\n3. Roland Fantom-8 - $248,500\n4. Tama Starclassic Bubinga 5-Piece Shell Pack - $245,000\n5. Music Man John Petrucci Majesty - $240,000\n6. Nord Stage 3 Compact - $227,500'

In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify, render_template, send_from_directory, make_response
from pyngrok import ngrok, conf
import requests
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# Initialize Flask app

conf.get_default().auth_token = "2fNSqOSH8CLG8ChWeVy9kOmRxyA_3K2XfGTAfMQ8btmZ869vZ"

app = Flask(__name__)

@app.route('/')
def index():
    # Load HTML content
    html_url = 'https://panosgio.org/hackathon/index.html'
    html_response = requests.get(html_url)
    html_content = html_response.text

    # Load CSS content
    css_url = 'https://panosgio.org/hackathon/ui.css'
    css_response = requests.get(css_url)
    css_content = css_response.text

    # Load JavaScript content
    js_url = 'https://panosgio.org/hackathon/ui.js'
    js_response = requests.get(js_url)
    js_content = js_response.text

    # Embed CSS and JS into HTML
    html_content = html_content.replace('<link rel="stylesheet" href="ui.css">', '<style>' + css_content + '</style>')
    html_content = html_content.replace('<script src="ui.js"></script>', '<script>' + js_content + '</script>')

    # Return the modified HTML content
    return make_response(html_content)

# Define the endpoint
@app.route('/run_query', methods=['POST'])
def run_query():
    # Extract question from the request
    question = request.json.get('question')

    # Your existing code to process the question
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # Placeholder for language model initialization
    generate_query = create_sql_query_chain(llm, db, k=1000)  # Assume this function exists
    query = generate_query.invoke({"question": question})

    # Use PromptTemplate to rephrase the answer
    answer_prompt = PromptTemplate.from_template("write it to sql format. : SQL Query: {query} Answer: ")
    rephrase_answer = answer_prompt | llm | StrOutputParser()

    # Use RunnablePassthrough to manage data flow
    execute_query = QuerySQLDataBaseTool(db=db)  # Placeholder for SQL execution
    chain = (
        RunnablePassthrough.assign(query=generate_query).assign(
            result=itemgetter("query") | execute_query
        )
        | rephrase_answer
    )

    query2 = chain.invoke({"question": question})

    execute_query = QuerySQLDataBaseTool(db=db)
    execute_query.invoke(query2)


    answer_prompt = PromptTemplate.from_template(
        """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
    )

    rephrase_answer = answer_prompt | llm | StrOutputParser()

    chain = (
        RunnablePassthrough.assign(query=generate_query).assign(
            result=itemgetter("query") | execute_query
        )
        | rephrase_answer
    )




    # Return the response
    return jsonify({"answer": chain.invoke({"question": question})})

# Run the Flask app on Colab
if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(' * Tunnel URL:', public_url)
    app.run(port=5000)


 * Tunnel URL: https://59fa-35-233-196-116.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:25:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:25:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:25:57] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:32:35] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:32:50] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:33:44] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:33:50] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:34:03] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:34:17] "POST /run_query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:34:31